In [1]:
# COMP30027 Machine learning Project 1: Gaining Information about Naive Bayes
# Author: Jordan Ung <jordanu@student.unimelb.edu.au> [729938]
# Last Modified: 22.03.19

#### GENERAL GUIDE
# Read In And Inspect The Data
# Check for missing value - (1) delete rows with missing values, (2) Impute the missing values with dataset
# Check for anomalously extreme values

In [3]:
# Import dependencies
import re

In [23]:
# Preprocess takes the name of a file and returns a list of instances
# within that file, with each instance containing a list of attributes
def preprocess(file_name):
    dataset = []
    with open(file_name, 'r') as file:
        # Add each instance to a list to be used later
        for line in file.readlines():
            dataset.append(line.strip().split(','))
    return dataset

data_file = 'anneal.csv'
# preprocess(data_file)

In [35]:
# Train takes a list of instances and returns a 2-tuple, containing
# a list of counts representing the class distribution and a list
# of lists of dictionaries representing the conditional probabilities
def train(instance_list):
    data_info = ([], [])
    classes = {}
    # TODO
    # Need to create a list of lists of dictionaries for repping the
    # conditional probabilities of each attribute w.r.t to the classes
    
    # Calculate the frequencies of all the class values
    for instance in instance_list:
        class_type = instance[-1]
        if class_type not in classes:
            classes[class_type] = 1
        else:
            classes[class_type] += 1
    
    # Find probability of each class value and create a list of
    # each class and input them into the 2-tuple
    for type in classes:
        data_info[0].append(classes[type] * 1.0 / len(instance_list))
        data_info[1].append([])
        
    for data in instance_list:
        
    
    return data_info# Return the 2-tuple
train(preprocess(data_file))

([0.7616926503340757,
  0.044543429844097995,
  0.008908685968819599,
  0.07461024498886415,
  0.11024498886414254],
 [[], [], [], [], []])

In [3]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(model, instance):
    return # Return a class

In [4]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate():
    return

In [5]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain():
    return

In [22]:
a = ([],[])
a[0].append(1)
print(a)




([1], [])
